In [1]:
import trimesh
from pathlib import Path
from sklearn.decomposition import PCA
import numpy as np
from numpy import array
from numpy import mean
from numpy import cov
from numpy.linalg import eig
from sklearn.decomposition import PCA

In [2]:
# attach to logger so trimesh messages will be printed to console
# trimesh.util.attach_to_log()

In [3]:
# mesh objects can be created from existing faces and vertex data
#mesh = trimesh.Trimesh(vertices=[[0, 0, 0], [0, 0, 1], [0, 1, 0]],
                       #faces=[[0, 1, 2]])

In [4]:
# by default, Trimesh will do a light processing, which will
# remove any NaN values and merge vertices that share position
# if you want to not do this on load, you can pass `process=False`
#mesh = trimesh.Trimesh(vertices=[[0, 0, 0], [0, 0, 1], [0, 1, 0]],
                       #faces=[[0, 1, 2]],
                       #process=False)

In [2]:
ID_0 = trimesh.load('ICT-FaceKit-master\FaceXModel\identity000.obj')

faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!


In [3]:
def as_mesh(scene_or_mesh):
    """
    Convert a possible scene to a mesh. The returned mesh has only vertex and face data.
    """
    if isinstance(scene_or_mesh, trimesh.Scene):
        if len(scene_or_mesh.geometry) == 0:
            mesh = None  # empty scene
        else:
            # we lose texture information here
            mesh = trimesh.util.concatenate(
                tuple(trimesh.Trimesh(vertices=g.vertices, faces=g.faces)
                    for g in scene_or_mesh.geometry.values()))
    else:
        assert(isinstance(mesh, trimesh.Trimesh))
        mesh = scene_or_mesh
    return mesh

In [4]:
ID_0_mesh = as_mesh(ID_0)

In [5]:
ID_0_mesh.remove_unreferenced_vertices()

In [33]:
#ID_0_mesh.euler_number

In [34]:
#print(ID_0_mesh.volume / ID_0_mesh.convex_hull.volume)

In [6]:
ID_0_mesh.show()

In [7]:
type(ID_0_mesh.vertices)

trimesh.caching.TrackedArray

In [8]:
ID_0_mesh_np_test = np.array(ID_0_mesh.vertices)
# turn trimesh tracked array into numpy array

In [9]:
ID_0_mesh_np_test = ID_0_mesh_np_test.flatten()
# flatten array into one column 

In [10]:
np.shape(ID_0_mesh_np_test)
paths = Path('C:\\Users\\judya\\Desktop\\AIFoundation\\ICT-FaceKit-master\\FaceXIDs').glob('**/*.obj')
len(list(paths))
ID_0_mesh_np_test

array([ 2.06112e+00,  3.52097e+00,  9.68861e+00, ..., -1.09969e-08,
        6.33494e+00,  1.05534e+01])

In [11]:
def get_shape_vectors():
    
    paths = Path('C:\\Users\\judya\\Desktop\\AIFoundation\\ICT-FaceKit-master\\FaceXIDs').glob('**/*.obj')
    #total_scans = len(list(paths))
    #data = np.empty([total_scans, 80793]) # (number of scanned faces) by (number of vertices * three dimensions)
    data = []
    for path in paths:
        path_in_str = str(path)
        scan_ID = trimesh.load(path_in_str)
        scan_ID = as_mesh(scan_ID)
        scan_ID = np.array(scan_ID.vertices)
        scan_ID = scan_ID.flatten()
        scan_ID = scan_ID.tolist()
        data.append(scan_ID)
    return data

In [12]:
shape_list = get_shape_vectors()

faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slow fallback!
faces have mixed data, using slo

In [13]:
for element in shape_list:
    print(len(element))
## The mesh data that I'm loading directly from the object files are not consistent

80793
80784
80784
80781
80790
80796
80796
80799
80787
80793
80799


In [14]:
# From the deeptwins resources files on uscict
ID_modes = np.load('C:\\Users\\judya\\Desktop\\AIFoundation\\ICT-FaceKit-master\\FaceXModel\\idendity_modes.npy')

In [15]:
np.shape(ID_modes)
#(xyz, number_of_vertices_in_narrow_face_area, number_of_faces)
# should be (number_of_faces, xyz * number_of_vertices)
ID_reshaped = np.reshape(ID_modes, (20118,100)) # this is wrong!!
np.shape(ID_reshaped)

(20118, 100)

In [16]:
#SKLEARN PCA

#create the PCA instance
pca = PCA(2) 
#fit on data
pca.fit(ID_reshaped)

#transform data
PrincipalComponents = pca.transform(ID_reshaped)
print(PrincipalComponents)
#np.shape(PrincipalComponents)

[[-0.02296584  0.02037357]
 [-0.02295963  0.02035197]
 [-0.02296208  0.02036111]
 ...
 [-0.16008385 -0.19039588]
 [-0.15812871 -0.13610064]
 [-0.16004152 -0.13944718]]


In [17]:
# NUMPY PCA

#calculate the mean of each column
column_mean = mean(ID_reshaped.T, axis=1)
#center columns by subtracting column means
center = ID_reshaped - column_mean
#calculate covariance matrix of centered matrix
covariance_matrix = cov(center.T)
#eigendecomposition of covariance matrix
values, vectors = eig(covariance_matrix)
#project data
PrincipalComponents = vectors.T.dot(center.T)
print(PrincipalComponents.T)
#np.shape(PrincipalComponents.T)

[[ 2.29658435e-02  2.03735685e-02 -2.88803353e-03 ... -2.18998668e-05
   1.07503924e-03  1.43205005e-03]
 [ 2.29596290e-02  2.03519719e-02 -2.88457624e-03 ... -3.36876284e-04
  -1.36554779e-04  1.95370787e-03]
 [ 2.29620762e-02  2.03611107e-02 -2.88601603e-03 ... -3.53096016e-04
   6.72416311e-04  1.75620862e-03]
 ...
 [ 1.60083845e-01 -1.90395878e-01  9.57755531e-02 ...  8.85278680e-04
  -1.52579963e-03 -2.53059230e-04]
 [ 1.58128707e-01 -1.36100644e-01  8.07511468e-02 ...  4.22523993e-05
  -1.58483492e-03  4.48067890e-04]
 [ 1.60041515e-01 -1.39447178e-01  8.43463906e-02 ... -6.21471264e-04
  -2.24365772e-03  8.40954719e-04]]


In [18]:
#Questions I had while trying to do this:
# What should I have done differently in trying to format the initial data? 
# What shape should the matrices that you perform PCA on be? How do you reshape the data correctly?
# How many principal components should you specify to represent the variance? Or should you just put a confidence level? 
# How can I ensure that all projected shape vectors are orthogonal? Do they need to be?
# Once you reduce the dimension of the shape-vectors 
# Is the creation of the identity coefficients the main thing that the morphable model does for deeptwins?

In [19]:
# Sources
# ICT FaceKit: https://github.com/ICT-VGL/ICT-FaceKit
# ICT FaceKit Video: https://www.youtube.com/watch?v=MlGkzFeyCYc
# 3DMM Paper: https://gravis.dmi.unibas.ch/publications/Sigg99/morphmod2.pdf
# How to implement PCA: https://machinelearningmastery.com/calculate-principal-component-analysis-scratch-python/
# Deeptwins Pipeline, models, and resources